## Create ENV

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres
psql -c "CREATE EXTENSION pg_walinspect;"

## Naming

### LSN

#### Structure

<img src="./helpers/WAL - LSN.png" alt="drawing" width="700"/>

#### Demo

In [ ]:
# Current LSN
psql -c "select pg_current_wal_insert_lsn();"

### File

#### Name Structure

<img src="./helpers/WAL - WAL File Name.png" alt="drawing" width="700"/>

## Physical Layout

- File of 16 MB (configurable)
- Divided to pages of 8 KB (so 2000 pages per segment)

<img src="./helpers/wal-physical-layout.png" alt="drawing" width="700"/>

## Demo

In [ ]:
# Current WAL File and LSN
psql -c "select pg_current_wal_insert_lsn() lsn, pg_walfile_name(pg_current_wal_insert_lsn()) wal_file_name;"

In [ ]:
# Last modified WAL file
ls -alt $PGDATA/pg_wal

In [ ]:
# Swith WAL File
psql -c "select pg_switch_wal();"
psql -c "select pg_walfile_name(pg_current_wal_insert_lsn());"

In [ ]:
# Last modified WAL file - new one
ls -alt $PGDATA/pg_wal

### Advanced

#### Change current LSN with create table

In [ ]:
-- Run in pg_admin !!
-- Changing the LSN with create table
DO $$
DECLARE start_state pg_lsn;
DECLARE after_create pg_lsn;
DECLARE start_create_diff text;
BEGIN
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	DROP TABLE IF EXISTS a;
	CREATE TABLE a(a int);
	COMMIT; -- Try to ommit

	SELECT pg_current_wal_insert_lsn()
	INTO after_create;

	SELECT pg_size_pretty(after_create - start_state)
	INTO start_create_diff;

	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Create LSN: %', after_create;
	RAISE NOTICE 'Create - Start DIFF size: %', start_create_diff;
END;
$$

#### WAL Records on INSERT

In [ ]:
-- Run in pg_admin !!
DO $$
DECLARE start_state pg_lsn;
DECLARE after_insert pg_lsn;
DECLARE start_insert_diff text;
DECLARE rec record;
BEGIN
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	INSERT INTO a VALUES (1);
	COMMIT;

	SELECT pg_current_wal_insert_lsn()
	INTO after_insert;

	SELECT pg_size_pretty(after_insert - start_state)
	INTO start_insert_diff;

	RAISE NOTICE '------- Summary -------';
	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Insert LSN: %', after_insert;
	RAISE NOTICE 'Insert - Create DIFF size: %', start_insert_diff;
	RAISE NOTICE 'WAL Records between Start and Insert:';
	FOR rec in 
		SELECT * 
		FROM pg_get_wal_records_info(start_state, after_insert)
	loop
		RAISE NOTICE '------- WAL Record -------';
		RAISE NOTICE 'Start LSN: %', rec.start_lsn;
		RAISE NOTICE 'End LSN: %', rec.end_lsn;
		RAISE NOTICE 'Prev LSN: %', rec.prev_lsn;
		RAISE NOTICE 'Record Type: %', rec.record_type;
		RAISE NOTICE 'Total Record Length In Bytes: %', rec.record_length + rec.main_data_length;
		RAISE NOTICE 'Block Reference: %', rec.block_ref;
		RAISE NOTICE 'Transaction ID: %', rec.xid;
	end loop;
END;
$$

#### Block Ref - What Is That?

In [ ]:
psql << EOM
    SELECT pg_relation_filepath FROM pg_relation_filepath('a');
    SELECT * FROM pg_tablespace;
EOM

#### WAL On Create

In [ ]:
-- Run in pg_admin !!
-- What actually happens when we create a new table
DO $$
DECLARE start_state pg_lsn;
DECLARE after_create pg_lsn;
DECLARE start_create_diff text;
DECLARE rec record;
DECLARE rel_name text;
BEGIN
	DROP TABLE IF EXISTS b;
	DROP TABLE IF EXISTS create_wal_records;
	SELECT pg_current_wal_insert_lsn()
	INTO start_state;

	CREATE TABLE b(a int);
	COMMIT;

	SELECT pg_current_wal_insert_lsn()
	INTO after_create;

	-- Change to not temp to look on the rows with simple select
	CREATE TEMP TABLE create_wal_records AS
	SELECT 
		*,
		SPLIT_PART(
			SUBSTRING(block_ref, '[0-9]\/[0-9]+\/[0-9]*'),
			'/',
			3
		)::oid rel_oid
	FROM pg_get_wal_records_info(start_state, after_create);

	SELECT pg_size_pretty(after_create - start_state)
	INTO start_create_diff;
	

	RAISE NOTICE '------- Summary -------';
	RAISE NOTICE 'Start State LSN: %', start_state;
	RAISE NOTICE 'After Create LSN: %', after_create;
	RAISE NOTICE 'Create - Start DIFF size: %', start_create_diff;

	RAISE NOTICE 'WAL Records between Start and Insert:';
	FOR rec in 
		SELECT wal.*, cl.relname 
		FROM 
			create_wal_records wal
			LEFT JOIN pg_class cl ON wal.rel_oid = cl.oid
	loop
		RAISE NOTICE '------- WAL Record -------';
		RAISE NOTICE 'Relation Name: %', rec.relname;
		RAISE NOTICE 'Start LSN: %', rec.start_lsn;
		RAISE NOTICE 'End LSN: %', rec.end_lsn;
		RAISE NOTICE 'Prev LSN: %', rec.prev_lsn;
		RAISE NOTICE 'Record Type: %', rec.record_type;
		RAISE NOTICE 'Total Record Length in bytes: %', rec.record_length + rec.main_data_length;
		RAISE NOTICE 'Block Reference: %', rec.block_ref;
		RAISE NOTICE 'Transaction ID: %', rec.xid;
	end loop;
END;
$$